In [49]:
from sklearn.model_selection import RepeatedStratifiedKFold 
from keras import models
from keras import layers
from keras import optimizers
import keras.applications
from keras.utils.np_utils import to_categorical

import cPickle as pickle
import numpy as np
import os
import sys
import time

sys.path.append('../EXP/')
import ClevelandMcGill as C


EXPERIMENT = 'Figure12'
DATASET = 4
FEATUREGENERATOR = 'RandomForest'
FRAMED = 'Framed'

#
#
#
print 'Running', EXPERIMENT, 'on dataset', DATASET, FRAMED, 'with', FEATUREGENERATOR


EPOCHS = 10
FRAMED_BOOL = (FRAMED == 'Framed')
NO_SPLITS = 10
NO_REPEATS = 4
BATCH_SIZE = 32

FEATUREGENERATORS = {'RandomForestSparse': 'RandomForestSparse',
                     'RandomForest': 'RandomForest',
                     'MLP': 'Just MLP',
                     'LeNet': 'LeNet',
                     'VGG19': keras.applications.VGG19,
                     'Xception': keras.applications.Xception,
                     'DenseNet201': keras.applications.DenseNet201}


#
#
# We store now a folder structure:
#   RESULTS/Experiment/Dataset/FeatureGenerator/Framed/EPOCHS_NOSPLITS_NOREPEATS_INDEX.p
#
# INDEX is the index of the current run
#
OUTPUT_DIRECTORY = os.path.join('../RESULTS_NEW/', EXPERIMENT, str(DATASET), FEATUREGENERATOR, FRAMED)
if not os.path.exists(OUTPUT_DIRECTORY):
  os.makedirs(OUTPUT_DIRECTORY)


#
#
# LOAD DATA
#
#
if EXPERIMENT == 'Figure12':
  if FEATUREGENERATOR == 'RandomForestSparse':
    X, y = C.Figure12.load(dataset=int(DATASET),preview=False, images=False)
  else:
    images, framed_images, labels = C.Figure12.load(dataset=int(DATASET),preview=False)

    if FRAMED_BOOL:
      X = framed_images
    else:
      X = images

    if FEATUREGENERATOR != 'RandomForest':
      X = np.stack((X,)*3, -1) # make sure we have RGB
      
    X = X.astype(np.float32)-.5 # normalize
    y = labels


Running Figure12 on dataset 4 Framed with RandomForest


In [51]:
X.shape

(100000, 100, 100)

In [52]:
# MLP case.. no feature generation here!
oshape = X.shape
# we now need to flatten the data
X = X.reshape(X.shape[0], oshape[1]*oshape[2])

In [46]:
X[0], y[0]

(array([[-0.5, -0.5, -0.5, ..., -0.5, -0.5, -0.5],
        [-0.5, -0.5, -0.5, ..., -0.5, -0.5, -0.5],
        [-0.5, -0.5, -0.5, ..., -0.5, -0.5, -0.5],
        ..., 
        [-0.5, -0.5, -0.5, ..., -0.5, -0.5, -0.5],
        [-0.5, -0.5, -0.5, ..., -0.5, -0.5, -0.5],
        [-0.5, -0.5, -0.5, ..., -0.5, -0.5, -0.5]], dtype=float32), False)

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [47]:
classifier = RandomForestClassifier(n_estimators=128)

In [53]:
classifier.fit(X[:67499],y[:67499])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=128, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [54]:
aaaa = classifier.predict(X[68000:])

In [55]:
np.array_equal(aaaa, y[68000:])

False

In [58]:
aaaa[0:20]

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True,  True, False,  True, False,
        True,  True], dtype=bool)

In [59]:
y[68000:][0:20]

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True,  True, False,  True, False,
        True,  True], dtype=bool)

In [62]:
len(np.where(aaaa != y[68000:])[0])

603

In [64]:
603/32000.

0.01884375